## Fake News Classification using RNN LSTM

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r'C:\Users\Shiva\Downloads\fake-news\train.csv')
#df=pd.read_csv(r'C:\Users\Shiva\Downloads\fake-news\test.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
#Drop Nan Values
df=df.dropna()

In [6]:
#Indepedendent Feature
X=df.drop('label',axis=1)
#Dependent Feature
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
messages=X.copy()

In [13]:
messages.reset_index(inplace=True)

In [17]:
#importing libarries related to word pre-processing
import nltk
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Pre-Processing

In [23]:
### Dataset Preprocessing
ps = PorterStemmer()
corpus = []
for index,value in messages.iterrows():
    review = re.sub('[^A-z]', ' ', value['title'])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hi

## One Hot Encoding

In [34]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [27]:
### Vocabulary size
voc_size=5000

In [28]:
onehot_repr=[one_hot(words,voc_size) for words in corpus] 
onehot_repr

[[3710, 4451, 689, 876, 4878, 2940, 4856, 4260, 3326, 2855],
 [2831, 313, 4484, 2255, 2800, 2338, 1136],
 [3829, 81, 2095, 876],
 [2769, 3563, 4647, 2088, 2920, 4971],
 [212, 2800, 2249, 63, 2021, 1726, 2800, 545, 4331, 1366],
 [3498,
  3175,
  4888,
  2113,
  1,
  1717,
  2368,
  4217,
  4001,
  2232,
  4140,
  2420,
  4758,
  4860,
  1136],
 [4690, 3008, 3123, 631, 2230, 215, 4751, 183, 3537, 4878, 4661],
 [4235, 20, 3415, 3064, 2506, 2218, 1717, 2741, 3537, 4878, 4661],
 [1172, 1259, 1770, 2699, 4696, 4773, 686, 1433, 1717, 2014],
 [1352, 3170, 1630, 1487, 4129, 2843, 1723, 553],
 [65, 3992, 4413, 3300, 3289, 2721, 2254, 1315, 3083, 4754, 4662],
 [2088, 2279, 4878, 4773, 1717, 2506],
 [2824, 922, 2229, 4376, 1793, 480, 1313, 1635, 1737],
 [755, 3503, 168, 3102, 4201, 1865, 2437, 3537, 4878, 4661],
 [2876, 3545, 4643, 2529, 92, 3537, 4878, 4661],
 [984, 2898, 2106, 585, 1247, 2215, 4583, 2207, 73, 1021],
 [1903, 4474, 313],
 [2199, 2749, 1578, 2385, 1717, 2104, 578, 1136],
 [1147, 31

## Word Embedding

In [31]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4260 3326 2855]
 [   0    0    0 ... 2800 2338 1136]
 [   0    0    0 ...   81 2095  876]
 ...
 [   0    0    0 ... 3537 4878 4661]
 [   0    0    0 ... 4083  278 4717]
 [   0    0    0 ... 3582  128 3409]]


In [32]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3710,
       4451,  689,  876, 4878, 2940, 4856, 4260, 3326, 2855])

## Model Creation

In [60]:
# Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)) #100 is the number of Neuron
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
len(embedded_docs),y.shape

(18285, (18285,))

In [62]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

## Model Training

In [64]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [66]:
# spliting validation size at 30%
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [85]:
#Final Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12799 samples, validate on 5486 samples
Epoch 1/10
12799/12799 [==============================] - 14s 1ms/sample - loss: 4.8136e-06 - acc: 1.0000 - val_loss: 0.9945 - val_acc: 0.9090
Epoch 2/10
12799/12799 [==============================] - 15s 1ms/sample - loss: 4.2717e-06 - acc: 1.0000 - val_loss: 0.9988 - val_acc: 0.9087
Epoch 3/10
12799/12799 [==============================] - 14s 1ms/sample - loss: 3.7949e-06 - acc: 1.0000 - val_loss: 1.0035 - val_acc: 0.9089
Epoch 4/10
12799/12799 [==============================] - 12s 962us/sample - loss: 3.3925e-06 - acc: 1.0000 - val_loss: 1.0067 - val_acc: 0.9092
Epoch 5/10
12799/12799 [==============================] - 11s 873us/sample - loss: 3.1095e-06 - acc: 1.0000 - val_loss: 1.0133 - val_acc: 0.9087
Epoch 6/10
12799/12799 [==============================] - 11s 871us/sample - loss: 2.7205e-06 - acc: 1.0000 - val_loss: 1.0163 - val_acc: 0.9083
Epoch 7/10
12799/12799 [==============================] - 11s 848us/sample - loss: 2.47

In [86]:
y_pred=model.predict_classes(X_test)

In [87]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
confusion_matrix(y_pred,y_test)

array([[2828,  224],
       [ 279, 2155]], dtype=int64)

In [88]:
accuracy_score(y_pred,y_test)

0.9083120670798396

In [89]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      3052
           1       0.91      0.89      0.90      2434

    accuracy                           0.91      5486
   macro avg       0.91      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486

